# Clustering

In clustering, the goal is to partition the set of inputs into disjoint groups, aka clusters.

It's similar to the classification problem but we have no prior information about the meaning of the groups.
We just want to group similar inputs together.

Hence, this problem requires unsupervised learning, as we don't have labelled training data.
(Or maybe we have but want to find out if there is a better grouping.)

## k-means clustering

Let's start with a simple case: we have 2-dimensional numerical data and know the number of clusters we want is 3.

In [10]:
from sklearn.datasets import make_blobs
import plotly.express as px

X, _ = make_blobs(cluster_std=0.8, random_state=0)
fig = px.scatter(x=X[:, 0], y=X[:, 1])
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()

### Intuition

How should we approach?

In classification, we used lines to separate the groups, and adjusted the position of the lines so the groups better match their labels.

Here, we don't have labels.
We can draw lines anywhere but how would we know which grouping is better?

Idea: Instead of separators, define center points (called centroids) for the groups, and assign inputs to the nearest group.

In [11]:
import numpy as np

centroids = np.array([(-2, 3), (1, 5), (3, 1)], dtype=np.float64)
distances = np.array([[np.linalg.norm(p - c) for c in centroids] for p in X])
print(distances[:5])
groups = np.argmin(distances, axis=1)
print(groups)

[[5.05444203 4.53034707 0.553246  ]
 [2.77667079 0.83503436 4.53477339]
 [5.30304097 4.59882666 0.29355571]
 [2.03095407 1.57463749 4.56153887]
 [2.14400209 2.02511403 3.59076034]]
[2 1 2 1 1 1 0 0 2 1 1 1 2 1 0 2 0 1 0 0 0 0 0 0 2 2 2 2 0 0 1 2 2 1 0 1 1
 2 2 0 0 2 2 1 1 1 2 2 0 0 1 2 1 2 0 0 2 2 1 2 2 0 0 0 0 2 1 0 2 1 0 2 0 2
 2 1 1 1 0 2 1 1 2 1 2 1 1 1 2 1 2 2 0 0 0 0 1 1 0 0]


In [12]:
from plotly import graph_objects as go

fig = px.scatter(
    x=X[:, 0],
    y=X[:, 1],
    color=groups,
    color_continuous_scale=px.colors.sequential.Bluered,
)
fig.add_scatter(
    x=centroids[:, 0],
    y=centroids[:, 1],
    mode="markers",
    marker=go.scatter.Marker({"symbol": "x", "color": "green"}),
)
fig.update_layout(showlegend=False)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()

Next step: Adjust the position of centroids.

Calculate the actual center points of the groups, and make them the new centroids.
These are actually the means of the groups, hence the name, k-means algorithm.

In [13]:
for i in range(len(centroids)):
    points = [x for j, x in enumerate(X) if groups[j] == i]
    centroids[i] = np.mean(points, axis=0)
print(centroids)
distances = np.array([[np.linalg.norm(p - c) for c in centroids] for p in X])
newgroups = np.argmin(distances, axis=1)
print(newgroups == groups)

[[-1.61546049  2.88948393]
 [ 0.86169471  4.28084157]
 [ 2.14220715  1.31621944]]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [14]:
fig = px.scatter(
    x=X[:, 0],
    y=X[:, 1],
    color=newgroups,
    color_continuous_scale=px.colors.sequential.Bluered,
)
fig.add_scatter(
    x=centroids[:, 0],
    y=centroids[:, 1],
    mode="markers",
    marker=go.scatter.Marker({"symbol": "x", "color": "green"}),
)
fig.update_layout(showlegend=False)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()

In [15]:
groups = newgroups
print(centroids)
for i in range(len(centroids)):
    points = [x for j, x in enumerate(X) if groups[j] == i]
    centroids[i] = np.mean(points, axis=0)
print(centroids)
distances = np.array([[np.linalg.norm(p - c) for c in centroids] for p in X])
newgroups = np.argmin(distances, axis=1)
print(newgroups == groups)

[[-1.61546049  2.88948393]
 [ 0.86169471  4.28084157]
 [ 2.14220715  1.31621944]]
[[-1.6938747   2.8324883 ]
 [ 0.78959558  4.25181726]
 [ 2.14220715  1.31621944]]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


The centroids will converge to their optimal position, which minimizes the average distance to the group members.
However, this is only a local optimum, depending on the initial centroid values.

One solution is to repeat the process with different starting positions.

But how to choose initial centroids? Previously, we've chosen them manually but when the number of dimensions is high, it becomes difficult to select good values.

Idea: Select k input points randomly as initial centroids.

In [20]:
k = 3
bestcentroids = None
bestgroups = None
bestinertia = None
# try 10 times
for _ in range(10):
    # initial centroids
    centroids = X[
        np.random.choice(X.shape[0], k, replace=False)
    ]  # with pandas: df.sample(n=k)
    distances = np.array([[np.linalg.norm(p - c) for c in centroids] for p in X])
    newgroups = np.argmin(distances, axis=1)
    groups = np.array([])
    while not np.array_equal(groups, newgroups):
        groups = newgroups
        for i in range(len(centroids)):
            points = [x for j, x in enumerate(X) if groups[j] == i]
            centroids[i] = np.mean(points, axis=0)
        distances = np.array([[np.linalg.norm(p - c) for c in centroids] for p in X])
        newgroups = np.argmin(distances, axis=1)
    # sum of distances from closest centroid
    inertia = np.sum(np.min(distances, axis=1))
    if bestcentroids is None or inertia < bestinertia:
        bestcentroids = centroids.copy()
        bestgroups = groups.copy()
        bestinertia = inertia
print(f"{bestinertia=}")
fig = px.scatter(
    x=X[:, 0],
    y=X[:, 1],
    color=bestgroups,
    color_continuous_scale=px.colors.sequential.Bluered,
)
fig.add_scatter(
    x=bestcentroids[:, 0],
    y=bestcentroids[:, 1],
    mode="markers",
    marker=go.scatter.Marker({"symbol": "x", "color": "green"}),
)
fig.update_layout(showlegend=False)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()

bestinertia=96.33503550534058


### Using scikit-learn

In [21]:
from sklearn.cluster import KMeans

k = 3
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(X)
print(kmeans.cluster_centers_)
print(kmeans.labels_)
print(kmeans.labels_ == bestgroups)
print(kmeans.inertia_)

[[-1.6938747   2.8324883 ]
 [ 2.14220715  1.31621944]
 [ 0.78959558  4.25181726]]
[1 2 1 2 2 2 0 0 1 2 2 2 1 2 0 1 0 2 0 0 0 0 0 2 1 1 1 1 0 0 2 1 1 2 0 2 2
 1 1 0 0 1 1 2 2 2 1 1 0 0 2 1 2 1 0 0 1 1 2 1 1 0 0 0 0 1 2 0 1 2 0 1 2 1
 1 2 2 2 0 1 2 2 1 2 1 2 2 2 1 2 1 1 0 0 0 0 2 2 0 0]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
113.9404779820813


So, what if we don't know how many clusters there should be?

Try with different k-values, and observe the drops in the inertia.
This is called the *elbow method*.

In [22]:
inertia = []
maxK = 50
for k in range(1, maxK):
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
# fig = px.line(y=inertia, x=range(1, maxK), markers="o")
fig = px.line(y=inertia * np.arange(1, maxK), x=range(1, maxK), markers="o")
fig.show()


In [23]:
kmeans = KMeans(n_clusters=4, n_init=10)
kmeans.fit(X)
fig = px.scatter(
    x=X[:, 0],
    y=X[:, 1],
    color=kmeans.labels_,
    color_continuous_scale=px.colors.sequential.Bluered,
)
fig.add_scatter(
    x=kmeans.cluster_centers_[:, 0],
    y=kmeans.cluster_centers_[:, 1],
    mode="markers",
    marker=go.scatter.Marker({"symbol": "x", "color": "green"}),
)
fig.update_layout(showlegend=False)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)
fig.show()